<h1>1-1. 성남시 노선번호 목록 가져오기</h1>
<h5>경기버스 포탈에서 성남시 버스노선(인허가, 경유) 중복제거한 csv가져오기</h5>
<h5>가져온 csv에서 일반버스만 추출</h5>

In [3]:
import pandas as pd
pd.set_option('display.max_rows', None)

# CSV 파일 읽기
df = pd.read_csv('성남시_노선번호목록.csv',encoding ='utf-8-sig')

# "관할관청" 열에서 '성남시'가 아닌 행을 삭제
df = df[df['관할관청'] == '성남시']

# "운행업체"열에서 경기고속인 버스 제외
df = df[df['운행업체']!='경기고속']

df['노선번호']=df['노선번호'].astype(str)
# 1. 노선번호가 4글자 이상인 행 삭제 (단, "33-1"과 "720-1A"는 제외) 4글자 이상은 전부 일반버스가 아닌 직행좌석버스라서 삭제
df = df[~((df['노선번호'].str.len() > 3) & (~df['노선번호'].isin(['33-1', '720-1A', '100출'])))]

# 2. 맨 마지막 행부터 위로 2행 포함해서 3행 삭제 중복이라 삭제
df = df[:-3]

normal_bus_list = df
print(normal_bus_list.head())

   순번 관할관청    운행업체  노선번호           기점          종점  인가거리  인가대수  출퇴근배차  평일배차  \
0   1  성남시  성남시내버스   100         사기막골  가락시장.가락시장역  32.1     1    150   160   
1   2  성남시  성남시내버스  100출  산성역.포레스티아동문  남위례역.창곡교차로  32.0     1      0     0   
2   3  성남시    대원버스   101          오리역     수서역5번출구  44.1    10     15    30   
3   4  성남시    대원버스   103      도촌동9단지앞      사당역(중)  61.0    11     18    40   
4   5  성남시  성남시내버스   200    도촌동행정복지센터     거여역5번출구  39.6     7     30    40   

  주말배차 주중상행첫차 주중상행막차 주말상행첫차 주말상행막차 주중하행첫차 주중하행막차 주말하행첫차 주말하행막차  비고  
0  160  05:10  16:40  05:10  16:40  06:00  17:40  06:00  17:40 NaN  
1    0  03:10  03:10  00:00  00:00  04:00  04:00    NaN    NaN NaN  
2   45  05:00  22:10  05:00  22:10  06:10  23:25  06:10  23:25 NaN  
3   40  05:00  21:50  05:00  21:50  06:10  23:20  06:10  23:20 NaN  
4   65  05:00  22:00  05:00  22:00  06:10  23:10  06:10  23:10 NaN  


<h1>1-2. 노선번호와 노선아이디를 API호출을 통해 매칭</h1>
<h5>공공데이터에 노선번호를 토대로 API호출하여 노선아이디와 매치</h5>

In [5]:
import requests
import xml.etree.ElementTree as ET

# API URL 및 서비스 키
url = 'http://apis.data.go.kr/6410000/busrouteservice/getBusRouteList'
service_key = ''

# 결과를 저장할 DataFrame
bus_to_id = pd.DataFrame(columns=['노선번호', '기점', '종점', '노선아이디', '지역명', '노선유형명'])

for idx, row in normal_bus_list.iterrows():
    bus_number = row['노선번호']
    start_point = row['기점']
    end_point = row['종점']

    # API 요청 파라미터 설정
    params = {
        'serviceKey': service_key,
        'keyword': str(bus_number)  # 노선번호를 문자열로 변환
    }

    # API 호출
    response = requests.get(url, params=params)

    # 응답 처리 (XML 처리)
    if response.status_code == 200:
        # XML 응답을 ElementTree로 파싱
        root = ET.fromstring(response.content)

        # 노선번호만을 기준으로 검색이라 다른지역 노선들도 전부 받아옴
        for bus_route in root.findall('.//busRouteList'):
            # id, 노선명, 지역명, 노선유형명만 뽑아옴
            route_id = bus_route.find('routeId').text if bus_route.find('routeId') is not None else None
            route_name = bus_route.find('routeName').text if bus_route.find('routeName') is not None else None
            region_name = bus_route.find('regionName').text if bus_route.find('regionName') is not None else None
            route_type_name = bus_route.find('routeTypeName').text if bus_route.find('routeTypeName') is not None else None

            # 노선번호가 정확히 일치하는 경우만 받기
            if route_name == bus_number:
                # DataFrame으로 추가할 데이터 생성
                new_data = pd.DataFrame([{
                    '노선번호': route_name,
                    '기점': start_point,
                    '종점': end_point,
                    '노선아이디': route_id,
                    '지역명': region_name,
                    '노선유형명': route_type_name
                }])

                # 기존 DataFrame과 새로운 데이터를 합침
                bus_to_id = pd.concat([bus_to_id, new_data], ignore_index=True)
#csv로 저장
bus_to_id.to_csv('bus_to_id.csv', index=False, encoding='utf-8-sig')
# 최종 결과 출력
print(bus_to_id.head())

  노선번호    기점          종점      노선아이디        지역명      노선유형명
0  100  사기막골  가락시장.가락시장역  234000873  구리,남양주,서울  직행좌석형시내버스
1  100  사기막골  가락시장.가락시장역  231000045         안성    일반형시내버스
2  100  사기막골  가락시장.가락시장역  228000396         용인    일반형시내버스
3  100  사기막골  가락시장.가락시장역  225000004      군포,의왕    일반형시내버스
4  100  사기막골  가락시장.가락시장역  215000030         연천    일반형시내버스


<h1>1-3. 지역명이 성남이면서 유형이 일반형시내버스인 노선아이디만 추출</h1>
<h5>성남시만 경유 노선 목록 데이터 추출 완료</h5>

In [6]:
seongnam_normal_bus_to_id = bus_to_id[
    bus_to_id['지역명'].str.contains('성남', case=False, na=False) &
    (bus_to_id['노선유형명'] == '일반형시내버스')
]
seongnam_normal_bus_to_id.reset_index(drop=True, inplace=True)
# 결과 출력
print(seongnam_normal_bus_to_id.head())

#seongnam_normal_bus_to_id.to_csv('seongnam_normal_bus_to_id.csv', index=False, encoding='utf-8-sig')

   노선번호           기점          종점      노선아이디             지역명    노선유형명
0   100         사기막골  가락시장.가락시장역  204000018           서울,성남  일반형시내버스
1  100출  산성역.포레스티아동문  남위례역.창곡교차로  204000083           서울,성남  일반형시내버스
2   101          오리역     수서역5번출구  228000179           서울,성남  일반형시내버스
3   103      도촌동9단지앞      사당역(중)  204000060  과천,서울,성남,안양,의왕  일반형시내버스
4   200    도촌동행정복지센터     거여역5번출구  204000029        서울,성남,하남  일반형시내버스


<h1>2-1 노선별 이용객 시계열 데이터 추출</h1>
<h5>경기도 교통 정보 데이터 센터에서 23년 성남시 노선별 이용객수 데이터 csv 추출</h5>
<h5>일반노선번호 리스트 기준으로 정제</h5>
<h6>평일 주말 구분하여 csv 저장</h6>

In [ ]:
#print(seongnam_normal_bus_to_id)
#23년 성남시 노선별 이용객수 데이터 가져옴
bus_time_client = pd.read_csv('23_성남시_노선별_시간대별_이용객수.csv', encoding='utf-8-sig')
bus_num_list = seongnam_normal_bus_to_id['노선번호'].tolist()
seongnam_normal_bus_time_client = bus_time_client[bus_time_client['노선'].isin(bus_num_list)]

print(seongnam_normal_bus_time_client.head())

#평일만
seongnam_normal_bus_time_client_weekday = seongnam_normal_bus_time_client[seongnam_normal_bus_time_client['일시'] == '평일']
#주말만
seongnam_normal_bus_time_client_weekend = seongnam_normal_bus_time_client[seongnam_normal_bus_time_client['일시'] == '주말']

seongnam_normal_bus_time_client_weekday.to_csv('seongnam_normal_bus_time_client_weekday.csv', index=False, encoding='utf-8-sig')
seongnam_normal_bus_time_client_weekend.to_csv('seongnam_normal_bus_time_client_weekend.csv', index=False, encoding='utf-8-sig')

print(seongnam_normal_bus_time_client_weekday.head())
print(seongnam_normal_bus_time_client_weekend.head())

   시/군/구    연도  월   노선              시종점  일시       시간  이용객수
92   성남시  2023  1  100  사기막골-가락시장.가락시장역  주말  05시~06시    16
93   성남시  2023  1  100  사기막골-가락시장.가락시장역  주말  06시~07시    20
94   성남시  2023  1  100  사기막골-가락시장.가락시장역  주말  07시~08시     2
95   성남시  2023  1  100  사기막골-가락시장.가락시장역  주말  08시~09시    34
96   성남시  2023  1  100  사기막골-가락시장.가락시장역  주말  09시~10시     6
    시/군/구    연도  월   노선              시종점  일시       시간  이용객수
106   성남시  2023  1  100  사기막골-가락시장.가락시장역  평일  05시~06시    22
107   성남시  2023  1  100  사기막골-가락시장.가락시장역  평일  06시~07시    54
108   성남시  2023  1  100  사기막골-가락시장.가락시장역  평일  07시~08시     7
109   성남시  2023  1  100  사기막골-가락시장.가락시장역  평일  08시~09시    83
110   성남시  2023  1  100  사기막골-가락시장.가락시장역  평일  09시~10시    12
   시/군/구    연도  월   노선              시종점  일시       시간  이용객수
92   성남시  2023  1  100  사기막골-가락시장.가락시장역  주말  05시~06시    16
93   성남시  2023  1  100  사기막골-가락시장.가락시장역  주말  06시~07시    20
94   성남시  2023  1  100  사기막골-가락시장.가락시장역  주말  07시~08시     2
95   성남시  2023  1  100  사기막골-가락시장.가락시장역  주말  08시~0

<h1>3-1 노선별 정류소 데이터 추출</h1>
<h5>공공데이터 센터 노선ID를 토대로 API호출</h5>

In [ ]:
# 서비스 키 설정 (서비스 키를 정확하게 넣어주세요)
service_key = ''

# 빈 DataFrame 생성 (결과를 저장할 DataFrame)
seongnam_bus_station = pd.DataFrame(columns=['노선번호', '노선아이디', '정류소아이디', '정류소순번', '정류소명', '정류소번호', 'x', 'y'])

# 노선아이디 목록 가져오기 (seongnam_normal_bus_to_id의 '노선아이디' 열)
bus_num_list = seongnam_normal_bus_to_id['노선아이디'].tolist()

# 각 노선아이디에 대해 API 호출
for route_id in bus_num_list:
    # API 요청 파라미터 설정
    params = {
        'serviceKey': service_key,
        'routeId': str(route_id)  # routeId는 문자열로 변환해서 전달
    }

    # API 호출
    response = requests.get('http://apis.data.go.kr/6410000/busrouteservice/getBusRouteStationList', params=params)

    # 응답 처리 (XML 처리)
    if response.status_code == 200:
        # XML 응답을 ElementTree로 파싱
        root = ET.fromstring(response.content)

        # 각 정류소 정보 추출
        for bus_station in root.findall('.//busRouteStationList'):
            # 필요한 값 추출 (정류소아이디, 정류소순번, 정류소명, 정류소번호, x, y)
            station_id = bus_station.find('stationId').text if bus_station.find('stationId') is not None else None
            station_seq = bus_station.find('stationSeq').text if bus_station.find('stationSeq') is not None else None
            station_name = bus_station.find('stationName').text if bus_station.find('stationName') is not None else None
            mobile_no = bus_station.find('mobileNo').text if bus_station.find('mobileNo') is not None else None
            x = bus_station.find('x').text if bus_station.find('x') is not None else None
            y = bus_station.find('y').text if bus_station.find('y') is not None else None

            # 노선번호와 노선아이디 가져오기 (seongnam_normal_bus_to_id에서 해당 노선번호, 노선아이디 추출)
            route_name = seongnam_normal_bus_to_id[seongnam_normal_bus_to_id['노선아이디'] == route_id]['노선번호'].values[0]
            route_id_value = route_id  # route_id는 그대로 사용

            # 새로운 데이터 생성
            new_data = pd.DataFrame([{
                '노선번호': route_name,
                '노선아이디': route_id_value,
                '정류소아이디': station_id,
                '정류소순번': station_seq,
                '정류소명': station_name,
                '정류소번호': mobile_no,
                'x': x,
                'y': y
            }])

            # 기존 DataFrame과 새로운 데이터를 합침
            seongnam_bus_station = pd.concat([seongnam_bus_station, new_data], ignore_index=True)

# 결과 출력
print(seongnam_bus_station.head())

# 최종 DataFrame을 CSV로 저장
seongnam_bus_station.to_csv('seongnam_bus_station.csv', index=False, encoding='utf-8-sig')

  노선번호      노선아이디     정류소아이디 정류소순번          정류소명   정류소번호            x  \
0  100  204000018  205000085     1          사기막골   06148  127.1794167   
1  100  204000018  205000033     2          영원무역   06135  127.1778833   
2  100  204000018  205000032     3  자동차검사소.산성아파트   06134  127.1780833   
3  100  204000018  205000038     4   궁전아파트.성지아파트   06146    127.17875   
4  100  204000018  205000037     5      근로자종합복지관   06139  127.1785167   

            y  
0  37.4445167  
1  37.4439333  
2  37.4416333  
3  37.4397167  
4  37.4383167  


<h1>4-1 정류소간 통과 노선 데이터</h1>
<h5>경기도 교통 정보 센터에서 데이터 추출</h5>

In [ ]:
pass_bus_station = pd.read_csv('23_성남시_정류소별_노선통과수.csv', encoding='utf-8-sig')
print(pass_bus_station.head())

     연도 관할지역 버스유형      정류소ID  정류소번호             정류소명  통과노선수     정류소아이디  \
0  2023  성남시   일반  206000535   7492   판교역.낙생육교.현대백화점     21  206000535   
1  2023  성남시   일반  206000236   7077       수내역.정자아이파크      4  206000236   
2  2023  성남시   일반  206000618   7560  이매촌한신.서현역.AK프라자      4  206000618   
3  2023  성남시   일반  205000368   6345          모란역5번출구     11  205000368   
4  2023  성남시   일반  206000518   7347         국군수도병원정문      1  206000518   

             정류소명2  정류소번호2         x좌표        y좌표   행정동             정류장명  \
0   판교역.낙생육교.현대백화점    7492  127.111700  37.391483   백현동   판교역.낙생육교.현대백화점   
1       수내역.정자아이파크    7077  127.110900  37.376433  수내1동       수내역.정자아이파크   
2  이매촌한신.서현역.AK프라자    7560  127.125833  37.386867  서현1동  이매촌한신.서현역.AK프라자   
3          모란역5번출구    6345  127.128967  37.430900   성남동          모란역5번출구   
4         국군수도병원정문    7347  127.149450  37.390717  서현1동         국군수도병원정문   

         상세위치                                        시내버스_경유노선번호 시외버스_경유노선번호  
0  판교휴먼시아5단지A  8201